In [1]:
import os
from datetime import datetime
from pathlib import Path

# import h5py
import scipy
import json
import pickle
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Duplicate key in file PosixPath('/home/lxh/miniconda3/envs/rul_prediction/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 268 ('font.family         : sans-serif        ')
Duplicate key in file PosixPath('/home/lxh/miniconda3/envs/rul_prediction/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 272 ('font.sans-serif: DejaVu Sans, Bitstream Vera Sans, Computer Modern Sans Serif, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif')


In [2]:
%matplotlib inline
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

In [4]:
def load_mat(file_path):
    filename = os.path.basename(file_path).split(".")[0] # eg.B0005
    data = scipy.io.loadmat(file_path)
    # print(type(data)) # dict
    # print(data.keys()) # dict_keys(['__header__', '__version__', '__globals__', 'B0005'])
    # print(data["B0005"][0][0][0][0].shape) # (616,)，实验持续了616个循环
    # print(type(data["B0005"][0][0][0][0][0])) # 用scipy读取mat文件，读取到结构体返回 numpy.void 类型数据

    cols = data[filename][0][0][0][0]
    
    data = []
    
    for col in cols:
        _ = {}
        parameter = {}
        keys = list(col[3][0][0].dtype.fields.keys())
        if str(col[0][0]) != "impedance":
            for idx, key in enumerate(keys):
                parameter[key] = list(col[3][0][0][idx][0])

        operation_type = str(col[0][0])
        temperature = str(col[1][0][0])
        time = str(datetime(
            int(col[2][0][0]), int(col[2][0][1]), int(col[2][0][2]), int(col[2][0][3]), int(col[2][0][4]), int(col[2][0][5])
        ))
        _["type"], _["temperature"], _["time"], _["data"] = operation_type, temperature, time, parameter
        data.append(_)

    return data

In [6]:
def extract_charge(data):
    charge_input = []
    bcycle = len(data) # 616
    print(bcycle)
    # 最后一组数据的测量点过少，舍弃
    for i in range(0, bcycle - 1):
        if data[i]["type"] == "charge":
            end = len(data[i]["data"]["Voltage_measured"])
            le = end % 10

            vTemp = np.array(data[i]["data"]["Voltage_measured"][0:end-le])
            vTemp = vTemp.reshape(int(len(vTemp)/10), -1, order="F")
            print("1: ", vTemp.shape)
            print(vTemp)
            vTemp = vTemp.mean(axis=0)
            print("2: ", vTemp.shape)
            print(vTemp)

            iTemp = np.array(data[i]["data"]["Current_measured"][0:end-le])
            iTemp = iTemp.reshape(int(len(iTemp)/10), -1, order="F")
            iTemp = iTemp.mean(axis=0)

            tTemp = np.array(data[i]["data"]["Temperature_measured"][0:end-le])
            tTemp = tTemp.reshape(int(len(tTemp)/10), -1, order="F")
            tTemp = tTemp.mean(axis=0)

            charge_input.append(np.hstack((vTemp, iTemp, tTemp)))

    charge_input = np.array(charge_input)
    # print(charge_input.shape)
    return charge_input

def extract_discharge(data):
    bcycle = len(data)
    capacity = []
    for i in range(0, bcycle):
        if data[i]["type"] == "discharge":
            capacity.append(data[i]["data"]["Capacity"][0])
    return capacity

def max_min_norm(charge_input, init, cap):
    print(charge_input.shape)
    
    r = np.max(charge_input, axis=0) - np.min(charge_input, axis=0)
    print("here")
    print(np.max(charge_input, axis=0))
    print(np.min(charge_input, axis=0))
    print(r)
    print("here")
    X = (charge_input - np.min(charge_input, axis=0)) / r

    comp = len(charge_input) - len(cap)
    y = [init for _ in range(comp)] + cap
    ym = np.min(y)
    yr = np.max(y) - np.min(y)
    y = (y - ym) / yr
    return X, y, ym, yr

def min_max_scaler(data):
    min, max = np.min(data), np.max(data)
    r = max - min
    data = (data - min) / r
    return  data, min, r

def de_min_max_scaler(data, min, r):
    return data * r + min